In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor


from auxiliary.prepare import *
#from auxiliary.table2 import *
#from auxiliary.table3 import *
from auxiliary.table4 import *
#from auxiliary.table6 import *
#from auxiliary.table_formula import *

In [2]:
data = pd.read_stata('data/Authority.dta')

#data for table2,3,5,6
data = basic_setting(data)

In [3]:
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [4]:
#keep the observation needed
df = data
df = df[((df['turin_co_sample']==1) | (df['turin_pr_sample']==1)) & ((df['post_experience']>=5)|(df['post_experience'].isnull()==True))  & ((df['pre_experience']>=5)|(df['pre_experience'].isnull()==True))& (df['missing']==0)]
df = df[(df['ctrl_pop_turin_co_sample']==1) | (df['ctrl_pop_turin_pr_sample']==1) | (df['ctrl_exp_turin_co_sample']==1) | (df['ctrl_exp_turin_pr_sample']==1) | (df['ctrl_pop_exp_turin_co_sample']==1) | (df['ctrl_pop_exp_turin_pr_sample']==1)]
df = df.reset_index()
df.shape

(2016, 94)

In [5]:
#re-construct trend-pa
id_auth_remained = df['id_auth'].unique()
id_auth_remained_df = pd.DataFrame({'id_auth': [], 'group_num': []})
for i in range(len(id_auth_remained)):
    id_auth_remained_df.loc[i,'id_auth'] = id_auth_remained[i]
    id_auth_remained_df.loc[i,'group_num'] = i+1

for i in range(len(df)):
    for j in range(len(id_auth_remained_df)):
        if df.loc[i, 'id_auth'] == id_auth_remained_df.loc[j, 'id_auth']:
            df.loc[i, 'id_auth_remained'] = j+1
id_auth_remained_dum = pd.get_dummies(df['id_auth_remained']).rename(columns=lambda x: 'id_auth_remained' + str(x))
df = pd.concat([df, id_auth_remained_dum],axis = 1)

#df['id_auth_remained'].value_counts()
#value checked

In [6]:
#re-contstruc trend-pa 시작
for i in range(len(id_auth_remained_dum.columns)):
    df['trend_pa_remained_'+str(i+1)] = 0
    for j in range(len(df)):
        if df.loc[j, id_auth_remained_dum.columns[i]]==1 and df.loc[j, 'authority_code']!=3090272 and df.loc[j, 'authority_code']!=3070001:
            df.loc[j,'trend_pa_remained_'+str(i+1)] = 1
    df.drop([id_auth_remained_dum.columns[i]],axis = 1)

In [16]:
c_outcomes=1
t = "turin_co_sample"
g = "ctrl_exp"
o = "discount"
i = 5

###First SPECIFICATION###
#setting for first specification df1
df1 = df
#work_category = 공백이나 nan값없어서 필터링 안함
df1 = df1[(df1[t]==1) & (df1[g +'_' + t]==1) & (df1['post_experience']>=i) & (df1['pre_experience']>=i)& (df1['post_experience'].isnull()==False) & (df1['pre_experience'].isnull()==False) & (df1['missing']==0) & (df1[o].isnull()==False) & (df1['fiscal_efficiency'].isnull()==False) & (df1['reserve_price'].isnull()==False)&(df1['municipality'].isnull()==False)]

df1 = df1.reset_index() #to use loc
df1 = df1.sort_values(by = 'authority_code', ascending = True)
#df1 value checked 1262

df1['ind'] = np.nan
for i in range(len(df1)):
    if i == 0:
        df1.loc[i, 'ind'] = 1
    else:
        if df1.loc[i, 'authority_code'] != df1.loc[i-1, 'authority_code']:
            df1.loc[i, 'ind'] = 1
#ind sum=15 checked

In [17]:
#그냥 확인임 안돌려도 됨
df1['authority_code'].value_counts()
df1['ind'].sum()
all_years = df1['year'].unique()
str(all_years[0])

'2005.0'

In [18]:
#create dummies for administration-year pairs 
all_years = df1['year'].unique()
all_authorities = df1['authority_code'].unique()
auth_year_reg_col = []
for auth in all_authorities:
    for yr in all_years:
        df1['auth_year_' + str(auth)+'_' + str(yr)] = 0
        auth_year_reg_col.append('auth_year_' + str(auth)+'_' + str(yr))
        df1.loc[(df1['year']==yr) & (df1['authority_code']==auth), 'auth_year_' + str(auth)+'_' + str(yr) ] = 1
#df1['auth_year_3.0_2002.0'].value_counts() =13 checked

In [19]:
##regression for first stage
#create dummies for work category
all_categories = df1['work_category'].unique()
#len(all_categories) = 30 checked
for cat in all_categories:
    df1['cat_'+cat] = 0
    df1.loc[df1['work_category']==cat, 'cat_'+cat] =1
#dummy_cat 안함 앞에 beta 필요해서
#df1['cat_OG03'].value_counts() checked

In [20]:
### Regression first stage 
#setting
work_dum = pd.get_dummies(df1['work_category']).rename(columns=lambda x: 'work_dum_' + str(x))
year_dum = pd.get_dummies(df1['year']).rename(columns=lambda x: 'year_dum_' + str(x))
auth_dum = pd.get_dummies(df1['authority_code']).rename(columns=lambda x: 'auth_dum_' + str(x))
dum_df = pd.concat([work_dum, year_dum, auth_dum],axis = 1)
#이렇게 해주고 부터 fe_reg_1 singular matrix 걸림
df1 = pd.concat([df1,dum_df],axis = 1)

work_list = list(work_dum.columns)
year_list = list(year_dum.columns)
auth_list = list(auth_dum.columns)

In [21]:
reg_col = []
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
for k in auth_list:
    reg_col.append(k)
            
exog_var = ['fpsb_auction','reserve_price','municipality','fiscal_efficiency']
exog = exog_var + reg_col 
exog.remove('year_dum_2000.0')

exog.remove('work_dum_OG01')
exog.remove('auth_dum_3.0')
exog.remove('auth_dum_1708.0')

#reg_col for co_sample, discount,ctrl_exp, fe_reg_1&2
#값은 다음
#exog = exog_var + reg_col
    
#1. reg
fe_reg_1 = mt.reg(df1, o, exog, cluster = 'auth_anno', addcons= True, check_colinear = True)
print(fe_reg_1)
#값 나와따

#2. reg
fe_reg_2 = mt.reg(df1, o, exog, cluster = 'authority_code',addcons= True, check_colinear = True)
print(fe_reg_2)
#얘도 나왔따

#3. reg
reg_col = auth_year_reg_col
for cat in all_categories:
    reg_col.append('cat_'+cat)
exog_var = ['reserve_price','municipality','fiscal_efficiency']
exog = exog_var + reg_col

exog.remove('auth_year_4.0_2000.0')
exog.remove('auth_year_6.0_2000.0') 
exog.remove('auth_year_16.0_2002.0')
exog.remove('auth_year_16.0_2003.0')
exog.remove('auth_year_16.0_2004.0')
exog.remove('auth_year_1246.0_2000.0')
exog.remove('cat_OS07')
exog.remove('fiscal_efficiency')

fe_reg_3 = mt.reg(df1, o, exog, cluster = 'auth_anno', addcons = True, check_colinear = True) #singular error
print(fe_reg_3)
#값나옴

Dependent variable:	discount
N:			1262
R-squared:		0.6395
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
                    coeff    se      t   p>t  CI_low CI_high
fpsb_auction       12.178 1.329  9.163 0.000   9.541  14.815
reserve_price       0.000 0.000  5.988 0.000   0.000   0.000
municipality       -7.233 1.930 -3.748 0.000 -11.062  -3.404
fiscal_efficiency  -2.611 4.586 -0.569 0.570 -11.709   6.486
work_dum_OG02       0.223 0.629  0.354 0.724  -1.024   1.470
work_dum_OG03      -0.560 0.543 -1.032 0.304  -1.638   0.517
work_dum_OG04      -4.715 3.311 -1.424 0.157 -11.283   1.853
work_dum_OG06       0.816 0.667  1.223 0.224  -0.508   2.140
work_dum_OG07       4.494 5.104  0.881 0.381  -5.632  14.620
work_dum_OG08       0.788 1.600  0.492 0.624  -2.386   3.961
work_dum_OG10       8.384 0.510 16.450 0.000   7.373   9.396
work_dum_OG11       5.676 1.066  5.325 0.000   3.561   7.791
work_dum_OG12       5.928 1.926  3.079 0.003   2

In [46]:
df1['dummy_cat'] = 0
#exog에 있는 것만으로 계싼하기
beta_cat_list = []
beta_list = []
for i in range(len(exog)):
    for cat in all_categories:
        if exog[i] == 'cat_'+cat:
            beta_cat_list.append(exog[i])
    for exo in exog_var:
        if exog[i] == exo:
            beta_list.append(exog[i])

if o == 'discount':
    discount_hat = fe_reg_3.yhat
    for i in range(len(df1)):
        for cat in beta_cat_list:
            df1.loc[i, 'discount_beta'] = discount_hat[i] - (df1.loc[i,'dummy_cat']-df1.loc[i,cat] * fe_reg_3.beta[cat])
            for exo in beta_list:
                df1.loc[i,'discount_beta'] = df1.loc[i,'discount_beta']- df1.loc[i,exo]*fe_reg_3.beta[exo]
elif o == 'delay_ratio':
    delay_ratio_hat = fe_reg_3.yhat
    for cat in all_categories:
        df1['delay_ratio_beta'] = delay_ratio_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] # -df1['municipality']*fe_reg_3.beta['municipality']
elif o == 'overrun_ratio':
    overrun_ratio_hat = fe_reg_3.yhat
    for cat in all_categories:
        df1['overrun_ratio_beta'] = overrun_ratio_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] # -df1['municipality']*fe_reg_3.beta['municipality']
else:
    days_to_award_hat = fe_reg_3.yhat
    for cat in all_categories:
        df1['days_to_award_beta'] =days_to_award_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] #-df1['municipality']*fe_reg_3.beta['municipality']
#worked

In [48]:
df1['discount_beta'].value_counts()

8.773208     2
31.246967    1
34.578647    1
11.573486    1
24.115179    1
            ..
27.758995    1
12.338564    1
22.687773    1
22.409561    1
26.360089    1
Name: discount_beta, Length: 1261, dtype: int64

In [49]:
df.shape

(2016, 165)

In [13]:
#create weigths - working well
nrep_s = df1.groupby(['authority_code','year']).size().unstack(level=1)
df1_nrep = pd.DataFrame(nrep_s)/len(df1)
df1['weights'] = np.nan
for auth in all_authorities:
    for yr in all_years:
        df1.loc[(df1['authority_code']==auth)&(df1['year']==yr),'weights'] = df1_nrep.loc[auth, yr]

In [15]:
#Keep only beta coefficients for state*year terms
collapse_list = [o +'_beta', 'authority_code', 'year', 'fpsb_auction', 'municipality', 'fiscal_efficiency', 'missing', 'turin_co_sample', 'weights'] + year_list + auth_list
collapse = df1.groupby(['auth_anno'])[collapse_list].mean()

KeyError: "Columns not found: 'discount_beta'"

In [196]:
df2 = collapse
df2 = df2.reset_index()
df2.columns
df2.shape

(101, 32)

In [197]:
df2['discount_beta'].value_counts()
len(df2[df2['discount_beta']>30].index)
#discount_beta 값 틀림

10

In [198]:
#Core conley-taber method
exog_var = ['fpsb_auction', 'municipality', 'fiscal_efficiency']
reg_col = []
reg_col_new = []

#reg_col.append(j)
for i in auth_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)

for k in reg_col:
    for j in df2.columns:
        if k ==j:
            reg_col_new.append(j)
            
exog = exog_var + reg_col_new


X = df2.loc[:,exog]
vif = calc_vif(X)

#delete from col list
for i in range(len(vif)):
    if np.isnan(vif.loc[i, 'VIF']) == True:
        reg_col.remove(vif.loc[i, 'variables'])

exog = exog_var + reg_col_new

exog.remove('year_dum_2000.0')
exog.remove('auth_dum_3.0')
exog.remove('auth_dum_1866.0')

wls = mt.reg(df2, o+'_beta', exog , cluster = 'auth_anno',addcons = True, awt_name = 'weights')
print(wls)
#값 다름 beta가 다르니깐 당연한거임

#predic res
df2['eta'] = wls.resid
df2['eta'] = df2['eta']+ df2['fpsb_auction']*wls.beta['fpsb_auction']

Dependent variable:	discount_beta
N:			101
R-squared:		0.9791
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
                    coeff    se      t   p>t  CI_low CI_high
fpsb_auction       12.946 1.537  8.422 0.000   9.896  15.996
municipality       10.937 2.269  4.821 0.000   6.436  15.438
fiscal_efficiency  -6.868 5.316 -1.292 0.199 -17.414   3.679
auth_dum_4.0       -0.413 2.417 -0.171 0.865  -5.208   4.382
auth_dum_6.0        0.811 2.206  0.367 0.714  -3.565   5.187
auth_dum_9.0       -2.301 2.352 -0.978 0.330  -6.968   2.365
auth_dum_16.0      -7.112 2.433 -2.923 0.004 -11.939  -2.286
auth_dum_20.0      -2.500 2.371 -1.054 0.294  -7.203   2.204
auth_dum_25.0      -2.563 2.089 -1.227 0.223  -6.706   1.581
auth_dum_30.0      -3.518 2.324 -1.514 0.133  -8.129   1.092
auth_dum_1246.0    -0.176 1.281 -0.137 0.891  -2.718   2.366
auth_dum_1708.0    -2.860 0.949 -3.015 0.003  -4.743  -0.978
auth_dum_1739.0     1.836 1.067  1.721 0.088

In [199]:
#Create tilde
df2 = df2.sort_values(by = 'year',ascending = True)
df2_wls= df2[(df2['authority_code']==3090272) | (df2['authority_code']==3070001)]
df2_wls = pd.DataFrame(df2_wls.groupby(['year'])['fpsb_auction'].mean())
for i in range(len(df2)):
    if df2.loc[i, 'authority_code']==3090272 or df2.loc[i, 'authority_code']==3070001:
        for j in list(df2_wls.index):
            if df2.loc[i, 'year'] == j:
                df2.loc[i,'djtga'] = df2_wls.loc[j, 'fpsb_auction']

df2_wls = pd.DataFrame(df2.groupby(['year'])['djtga'].sum())
for i in range(len(df2)):
    for j in list(df2_wls.index):
        if df2.loc[i, 'year'] == j:
            df2.loc[i,'djt'] = df2_wls.loc[j, 'djtga']

df2 = df2.sort_values(by = 'authority_code', ascending = True)
df2_wls = pd.DataFrame(df2.groupby(['authority_code'])['djt'].mean())
for i in range(len(df2)):
    for j in list(df2_wls.index):
        if df2.loc[i, 'authority_code'] == j:
            df2.loc[i,'meandjt'] = df2_wls.loc[j, 'djt']

df2['dtil'] = df2['djt'] - df2['meandjt']
#df2['meandjt'] = df2['djt'].mean()
#df2['dtil'] = df2['djt'] - df2['meandjt']
#df2['dtil'].value_counts() #0밖에 없다구요..

In [200]:
df2['dtil'].value_counts()

 0.428571    52
-0.571429    39
 0.333333     4
-0.666667     2
-0.500000     2
 0.500000     2
Name: dtil, dtype: int64

In [201]:
#obtain diff in diff coeff
#renormalize weights
df2.loc[(df2['authority_code']==3090272) | (df2['authority_code']==3070001),'tot_weights'] = df2['weights'].sum()
df2['new_weights'] = df2['weights']/df2['tot_weights']
df2_wls = df2[(df2['authority_code']==3090272) | (df2['authority_code']==3070001)]
wls_2 = mt.reg(df2_wls, 'eta' , 'dtil' , awt_name = 'new_weights', addcons = True,check_colinear = True)
print(wls_2)
#얼추맞음
alpha = [wls_2.beta['dtil']] 
df2 = df2.drop(['tot_weights','new_weights'],axis = 1)

Dependent variable:	eta
N:			7
R-squared:		0.8263
Estimation method:	OLS
VCE method:		Standard (Homosk.)
       coeff    se     t   p>t CI_low CI_high
dtil  12.870 2.287 5.627 0.002  6.991  18.750
_cons  7.090 1.170 6.062 0.002  4.084  10.097



In [202]:
alpha

[12.87025753549585]

In [203]:
#simulataneous for each public
asim = []
'''auth_wls_3 = []
for auth in all_authorities:
    for i in range(len(df2)):
        if df2.loc[i, 'authority_code'] == auth:
            auth_wls_3.append(auth)'''
            
for auth in all_authorities:
    if auth !=3090272 and auth !=3070001:
        df2.loc[df2['authority_code']==auth, 'tot_weights'] = df2['weights'].sum()
        df2['new_weights'] = df2['weights']/df2['tot_weights']
        df2_wls_3 = df2[df2['authority_code']==auth]
        wls_3 = mt.reg(df2_wls_3, 'eta' , 'dtil' , awt_name = 'new_weights')
        asim.append(wls_3.beta['dtil'])
        df2 = df2.drop(['tot_weights','new_weights'],axis = 1)

#asim 이랑 alpha 길이 맞춰주기
for i in range(len(asim)-1):
    alpha.append(alpha[0])

asim_tmp = []
for i in range(min(len(alpha),len(asim))):
    asim_tmp.append(alpha[i] - asim[i])

asim = asim_tmp
df2['ci'] = np.nan
df2['asim'] = np.nan
for i in range(len(asim)):
    df2.loc[i, 'ci'] = asim[i]
    df2.loc[i, 'asim'] = asim[i]

#form confidence level
numst=len(asim)+1
i025=math.floor(0.025*(numst-1))
i025=max([i025,1])
i975=math.ceil(0.975*(numst-1))
i05=math.floor(0.050*(numst-1))
i05=max([i05,i025+1])
i95=math.ceil(0.950*(numst-1))
i95=min([i95,numst-2])

stima_ta = alpha[0]
df2.sort_values(by = 'asim',ascending = True)
ci_ta025 = min([df2.loc[i025,'ci'], df2.loc[i975, 'ci'] ])
ci_ta975 = max([df2.loc[i025,'ci'], df2.loc[i975, 'ci'] ])

#wls_4 setting
reg_col = []
for i in year_list:
    reg_col.append(i)
for j in auth_list:
    reg_col.append(j)
exog_var = ['fpsb_auction','municipality','fiscal_efficiency']
exog = reg_col+exog_var

exog.remove('year_dum_2000.0')
exog.remove('auth_dum_3.0')
exog.remove('auth_dum_1708.0')

wls_4 = mt.reg(df2, o+'_beta', exog, awt_name = 'weights', cluster = 'authority_code', addcons = True)
print(wls_4)
#얼추나옴
#end of first specification

Dependent variable:	discount_beta
N:			101
R-squared:		0.9791
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  authority_code
  No. of clusters:	  15
                    coeff    se       t   p>t  CI_low CI_high
year_dum_2001.0     2.022 1.583   1.277 0.222  -1.374   5.417
year_dum_2002.0     0.239 1.543   0.155 0.879  -3.071   3.549
year_dum_2003.0    -0.663 1.465  -0.452 0.658  -3.805   2.479
year_dum_2004.0     1.336 1.415   0.945 0.361  -1.698   4.371
year_dum_2005.0     3.616 1.391   2.600 0.021   0.633   6.600
year_dum_2006.0     3.664 1.838   1.993 0.066  -0.279   7.606
auth_dum_4.0       -0.413 0.361  -1.144 0.272  -1.187   0.361
auth_dum_6.0        0.811 0.463   1.751 0.102  -0.182   1.803
auth_dum_9.0       -2.301 0.716  -3.216 0.006  -3.836  -0.766
auth_dum_16.0      -7.112 0.709 -10.031 0.000  -8.633  -5.592
auth_dum_20.0      -2.500 0.216 -11.596 0.000  -2.962  -2.037
auth_dum_25.0      -2.563 0.283  -9.059 0.000  -3.169  -1.956
auth_dum_30.0      -3.518 0

In [185]:
len(alpha)

27

In [204]:
ci_ta025

10.105632776778588

In [205]:
ci_ta975

10.105632776778588